# Submit geometry optimization with SIESTA

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
yaml_text = """
standard_psml:
  description: 'A standard list of inputs for Siesta. Performaces tested on crystal elements giving an average Delta of 7.1 meV. No support for Spin-Orbit. Use of PSML pseudos, not supported by Siesta 4.0.*/4.1.* versions'
  parameters:
    xc-functional: "GGA"
    xc-authors: "PBE"
    max-scf-iterations: 50
    scf-mixer-history: 5
    scf-mixer-weight: 0.1
    scf-dm-tolerance: 1.e-4  #1.e-5
    solution-method: 'diagon'
    electronic-temperature: '25 meV'
    write-forces: True
    mesh-cutoff: '200 Ry'
  spin_additions:
    write-mulliken-pop: 1
  relax_additions:
    scf-dm-tolerance: 1.e-4
    md-max-force-tol: '0.04 eV/ang'
    md-max-stress-tol: '0.1 GPa'
  basis:
    pao-energy-shift: '50 meV'
    pao-basis-size: 'DZP'
  pseudo_family: 'PseudoDojo/0.4/PBE/SR/standard/psml'
  kpoints:
    distance: 0.1 #0.062
    offset: [0., 0., 0.]
  atomic_heuristics:
    Li:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Be:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Na:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Mg:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Mn:
      parameters:
        mesh-cutoff: '400 Ry'
    Fe:
      parameters:
        mesh-cutoff: '400 Ry'
    Ag:
      parameters:
        mesh-cutoff: '300 Ry'
    Ca:
      basis:
        pao-block: "Ca 3 \n  n=3   0   1 \n  3.505 \n n=4   0   2  \n 7.028      0.000 \n n=3   1   1 \n 4.072"
        split-tail-norm: True
    Sr:
      basis:
        pao-block: "Sr 3 \n  n=4   0   1 \n  3.809 \n n=5   0   2  \n  7.599      0.000  \n n=4   1   1 \n 4.538"
        split-tail-norm: True
    Ba:
      basis:
        pao-block: "Ba 3 \n  n=5   0   1 \n  4.369 \n n=6   0   2  \n 7.602      0.000 \n n=5   1   1 \n 5.205"
        split-tail-norm: True
    Sb:
      parameters:
        mesh-cutoff: '400 Ry'
    Hg:
      basis:
        pao-block: "Hg 4 \n  n=5   0   1 \n  3.568 \n n=6   0   2  \n 6.573  0.0 \n n=5   1   2 \n 4.059  0.0 \n n=5   2  1 \n 5.918"
standard_psf:
  description: 'copied from standard_psml but using PSF pseudos'
  parameters:
    xc-functional: "GGA"
    xc-authors: "PBE"
    max-scf-iterations: 50
    scf-mixer-history: 5
    scf-mixer-weight: 0.1
    scf-dm-tolerance: 1.e-4  #1.e-5
    solution-method: 'diagon'
    electronic-temperature: '25 meV'
    write-forces: True
    mesh-cutoff: '200 Ry'
  spin_additions:
    write-mulliken-pop: 1
  relax_additions:
    scf-dm-tolerance: 1.e-4
    md-max-force-tol: '0.04 eV/ang'
    md-max-stress-tol: '0.1 GPa'
  basis:
    pao-energy-shift: '50 meV'
    pao-basis-size: 'DZP'
  pseudo_family: 'psf_family'
  kpoints:
    distance: 0.1 #0.062
    offset: [0., 0., 0.]
  atomic_heuristics:
    Li:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Be:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Na:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Mg:
      parameters:
        mesh-cutoff: '250 Ry'
      basis:
        polarization: 'non-perturbative'
    Mn:
      parameters:
        mesh-cutoff: '400 Ry'
    Fe:
      parameters:
        mesh-cutoff: '400 Ry'
    Ag:
      parameters:
        mesh-cutoff: '300 Ry'
    Ca:
      basis:
        pao-block: "Ca 3 \n  n=3   0   1 \n  3.505 \n n=4   0   2  \n 7.028      0.000 \n n=3   1   1 \n 4.072"
        split-tail-norm: True
    Sr:
      basis:
        pao-block: "Sr 3 \n  n=4   0   1 \n  3.809 \n n=5   0   2  \n  7.599      0.000  \n n=4   1   1 \n 4.538"
        split-tail-norm: True
    Ba:
      basis:
        pao-block: "Ba 3 \n  n=5   0   1 \n  4.369 \n n=6   0   2  \n 7.602      0.000 \n n=5   1   1 \n 5.205"
        split-tail-norm: True
    Sb:
      parameters:
        mesh-cutoff: '400 Ry'
    Hg:
      basis:
        pao-block: "Hg 4 \n  n=5   0   1 \n  3.568 \n n=6   0   2  \n 6.573  0.0 \n n=5   1   2 \n 4.059  0.0 \n n=5   2  1 \n 5.918"

"""

In [ ]:
# General imports.
import urllib.parse as urlparse
import yaml
from copy import deepcopy

import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
# AiiDAlab imports.
import aiidalab_widgets_base as awb
import aiidalab_widgets_empa as awe
from aiida import orm, plugins

from surfaces_tools.utils import wfn

# Custom imports.
from surfaces_tools.widgets import editors, inputs


#Not required by AiiDA
import os.path as op
import sys

#AiiDA classes and functions
from aiida.engine import submit
from aiida.orm import Dict, KpointsData, StructureData, load_code
from aiida.tools import get_explicit_kpoints_path
from aiida_pseudo.data.pseudo.psf import PsfData
from aiida_siesta.workflows.base import SiestaBaseWorkChain

In [ ]:
# Structure selector.
build_slab = editors.BuildSlab(title="Build slab")
input_details = inputs.InputDetails()

structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.SmilesWidget(title="From SMILES"),
        awe.CdxmlUploadWidget(title="CDXML"),
    ],
    editors=[
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab,
        awb.BasicCellEditor(),
        editors.InsertStructureWidget(title="Insert molecule"),
    ],
    storable=True,
    node_class="StructureData",
)
ipw.dlink((structure_selector, "structure"), (build_slab, "molecule"))
ipw.dlink((structure_selector, "structure"), (input_details, "structure"))
ipw.dlink((input_details, "details"), (build_slab, "details"))
display(structure_selector)

# Code.
code_input_widget = awb.ComputationalResourcesWidget(
    description="SIESTA code:", default_calc_job_plugin="siesta"
)
resources = awe.ProcessResourcesWidget()

output = ipw.Output()

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
parsed_url = urlparse.parse_qs(url.query)
if "structure_uuid" in parsed_url:
    structure_selector.input_structure = load_node(parsed_url["structure_uuid"][0])

In [ ]:
# # Protocol.
# protocol = ipw.Dropdown(
#     value="standard_psf",
#     options=[
#         ("PSF", "standard_psf"),
#         ("PSML", "standard_psml"),
#     ],
#     description="Protocol:",
#     style={"description_width": "initial"},
# )

In [ ]:
import json
import ipywidgets as ipw
import yaml

# Load YAML
data = yaml.safe_load(yaml_text)
protocol_names = list(data.keys())

# -------------------------------------------------------------
# Utility: get element symbols from structure
# -------------------------------------------------------------
def get_elements_from_structure(structure_node):
    try:
        return sorted({site.kind_name for site in structure_node.sites})
    except Exception:
        return []

# -------------------------------------------------------------
# Create widget for a single value
# -------------------------------------------------------------
def make_widget(fullname, value):
    if isinstance(value, bool):
        w = ipw.Checkbox(description=fullname, value=value)
    elif isinstance(value, (int, float)):
        w = ipw.FloatText(description=fullname, value=value)
    elif isinstance(value, list):
        w = ipw.Textarea(description=fullname, value=json.dumps(value))
    else:
        w = ipw.Text(description=fullname, value=str(value))

    w._original_value = value
    w._user_modified = False

    def handler(change, widget=w):
        if change["name"] == "value":
            widget._user_modified = True

    w.observe(handler)
    return w

# -------------------------------------------------------------
# Turn a nested dict into a list of widgets
# -------------------------------------------------------------
def widgets_from_dict(dct, prefix=""):
    widgets = []
    for key, val in dct.items():
        fullname = f"{prefix}{key}" if prefix else key

        if isinstance(val, dict):
            widgets.extend(widgets_from_dict(val, prefix=f"{fullname}."))
        else:
            widgets.append(make_widget(fullname, val))

    return widgets

# -------------------------------------------------------------
# Convert section into VBox (with optional element filtering)
# -------------------------------------------------------------
def section_to_vbox(section_data, allowed_elements=None):
    if isinstance(section_data, dict):
        if allowed_elements is not None:
            # filter atomic_heuristics
            section_data = {
                el: section_data[el]
                for el in section_data.keys()
                if el in allowed_elements
            }
        widgets = widgets_from_dict(section_data)
        return ipw.VBox(widgets)

    # scalar section (e.g. pseudo_family)
    w = make_widget("", section_data)
    return ipw.VBox([w])

# -------------------------------------------------------------
# The main accordion update
# -------------------------------------------------------------
accordion = ipw.Accordion(children=[])

def update_accordion(*args):
    struct = getattr(structure_selector, "structure_node", None)

    if struct is None:
        accordion.children = []
        return

    protocol_key = protocol.value
    proto = data[protocol_key]
    elements = get_elements_from_structure(struct)

    tab_children = []
    tab_titles = []

    for section_name, section_content in proto.items():

        # skip YAML description
        if section_name == "description":
            continue

        # atomic heuristics → keep only present species
        if section_name == "atomic_heuristics":
            box = section_to_vbox(section_content, allowed_elements=elements)
            if not box.children:
                continue
        else:
            box = section_to_vbox(section_content)

        tab_children.append(box)
        tab_titles.append(section_name)

    # Create Tab widget
    tabs = ipw.Tab(children=tab_children)
    for i, title in enumerate(tab_titles):
        tabs.set_title(i, title)

    # Wrap tabs in a single accordion called "Advanced settings"
    accordion.children = [tabs]
    accordion.set_title(0, "Advanced settings")


# -------------------------------------------------------------
# Protocol dropdown
# -------------------------------------------------------------
protocol = ipw.Dropdown(
    value=protocol_names[0],
    options=[(p, p) for p in protocol_names],
    description="Protocol:",
    style={"description_width": "initial"},
)

protocol.observe(update_accordion, names="value")
structure_selector.observe(update_accordion, names="structure_node")




In [ ]:
workflow_description = ipw.Text(
    description="Workflow description:",
    placeholder="Provide the description here.",
    style={"description_width": "initial"},
    layout={"width": "70%"},
)

In [ ]:
ipw.dlink((code_input_widget, "value"), (input_details, "selected_code"))


def prepare_geometry_optimization():
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, "Select a structure first."
    elif not code_input_widget.value:
        can_submit, msg = False, "Select SIESTA code."
    else:
        calc_engines = {
            'siesta': {
                'code': code_input_widget.value,
                'options': {
                    'resources': {
                        'num_machines': resources.nodes, 
                        'num_mpiprocs_per_machine': resources.tasks_per_node,
                        'num_cores_per_mpiproc': resources.threads_per_task,
                    },
             "max_wallclock_seconds": resources.walltime_seconds, #'queue_name': 'DevQ', 'withmpi': True, 'account': "tcphy113c"
         }}}
        inp_gen = SiestaBaseWorkChain.inputs_generator()
        builder = inp_gen.get_filled_builder(structure_selector.structure_node, calc_engines, protocol.value)
        builder.metadata.description = workflow_description.value
        can_submit=True
        
    if not can_submit:
        with output:
            print(msg)
            return


    return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(
    SiestaBaseWorkChain,
    inputs_generator=prepare_geometry_optimization,
    disable_after_submit=False,
    append_output=True,
)

# Inputs

In [ ]:
#display(protocol)
ui = ipw.VBox([protocol, accordion])
ui

# Code and resources

In [ ]:
display(code_input_widget, resources)

# Submit

In [ ]:
display(workflow_description, btn_submit, output)